In [1]:
import numpy as np
import pandas as pd

In [2]:
import keras
print(keras.__version__)

2025-03-22 19:23:25.831122: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-22 19:23:25.832314: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 19:23:25.835884: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 19:23:25.844397: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-22 19:23:25.858361: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

3.7.0


In [3]:
# If you are using a version of Keras lower than 3, you will need to uncomment the following line.
# !pip install keras --upgrade

In [4]:
import keras
print(keras.__version__)

3.7.0


In [5]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [7]:
# Load the train and test data
train = pd.read_csv('./data/petfinder_train.csv')
test =  pd.read_csv('./data/petfinder_test.csv')

In [8]:
train.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           10537 non-null  object
 1   Age            10537 non-null  int64 
 2   Breed1         10537 non-null  object
 3   Gender         10537 non-null  object
 4   Color1         10537 non-null  object
 5   Color2         10537 non-null  object
 6   MaturitySize   10537 non-null  object
 7   FurLength      10537 non-null  object
 8   Vaccinated     10537 non-null  object
 9   Sterilized     10537 non-null  object
 10  Health         10537 non-null  object
 11  Fee            10537 non-null  int64 
 12  Description    10528 non-null  object
 13  PhotoAmt       10537 non-null  int64 
 14  AdoptionSpeed  10537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.2+ MB


In [10]:
dict = {
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 0
}

train['Target']= train['AdoptionSpeed'].replace(dict)

In [11]:
train.drop(columns = ['AdoptionSpeed','Description'], inplace = True)
test.drop(columns = ['Description'], inplace = True)

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          10537 non-null  object
 1   Age           10537 non-null  int64 
 2   Breed1        10537 non-null  object
 3   Gender        10537 non-null  object
 4   Color1        10537 non-null  object
 5   Color2        10537 non-null  object
 6   MaturitySize  10537 non-null  object
 7   FurLength     10537 non-null  object
 8   Vaccinated    10537 non-null  object
 9   Sterilized    10537 non-null  object
 10  Health        10537 non-null  object
 11  Fee           10537 non-null  int64 
 12  PhotoAmt      10537 non-null  int64 
 13  Target        10537 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 1.1+ MB


In [13]:
for column in ["Type", "Breed1", "Gender", "Color1", "Color2", "MaturitySize", "FurLength", "Vaccinated", "Sterilized", "Health"]:
    print("column:", column, "unique_values:", len(train[column].unique()))

column: Type unique_values: 2
column: Breed1 unique_values: 165
column: Gender unique_values: 2
column: Color1 unique_values: 7
column: Color2 unique_values: 7
column: MaturitySize unique_values: 3
column: FurLength unique_values: 3
column: Vaccinated unique_values: 3
column: Sterilized unique_values: 3
column: Health unique_values: 3


In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train['MaturitySize'] = le.fit_transform(train['MaturitySize'])
test['MaturitySize'] = le.transform(test['MaturitySize'])

train['FurLength'] = le.fit_transform(train['FurLength'])
test['FurLength'] = le.transform(test['FurLength'])

train['Health'] = le.fit_transform(train['Health'])
test['Health'] = le.transform(test['Health'])

In [15]:
train['Type'] = le.fit_transform(train['Type'])
test['Type'] = le.transform(test['Type'])

train['Gender'] = le.fit_transform(train['Gender'])
test['Gender'] = le.transform(test['Gender'])

In [16]:
!pip install category_encoders

In [17]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder()

train_binary_Breed1 = binary_encoder.fit_transform(train['Breed1'])
test_binary_Breed1 = binary_encoder.transform(test['Breed1'])

train_binary_Color1 = binary_encoder.fit_transform(train['Color1'])
test_binary_Color1 = binary_encoder.transform(test['Color1'])

train_binary_Color2 = binary_encoder.fit_transform(train['Color2'])
test_binary_Color2 = binary_encoder.transform(test['Color2'])

train_binary_Vaccinated = binary_encoder.fit_transform(train['Vaccinated'])
test_binary_Vaccinated = binary_encoder.transform(test['Vaccinated'])

train_binary_Sterilized = binary_encoder.fit_transform(train['Sterilized'])
test_binary_Sterilized = binary_encoder.transform(test['Sterilized'])

In [18]:
train = pd.concat([train, train_binary_Breed1, train_binary_Color1, train_binary_Color2, train_binary_Vaccinated, train_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
train.drop(columns = columns, inplace = True)

test = pd.concat([test, test_binary_Breed1, test_binary_Color1, test_binary_Color2, test_binary_Vaccinated, test_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
test.drop(columns = columns, inplace = True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10537 entries, 0 to 10536
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Type          10537 non-null  int64
 1   Age           10537 non-null  int64
 2   Gender        10537 non-null  int64
 3   MaturitySize  10537 non-null  int64
 4   FurLength     10537 non-null  int64
 5   Health        10537 non-null  int64
 6   Fee           10537 non-null  int64
 7   PhotoAmt      10537 non-null  int64
 8   Target        10537 non-null  int64
 9   Breed1_0      10537 non-null  int64
 10  Breed1_1      10537 non-null  int64
 11  Breed1_2      10537 non-null  int64
 12  Breed1_3      10537 non-null  int64
 13  Breed1_4      10537 non-null  int64
 14  Breed1_5      10537 non-null  int64
 15  Breed1_6      10537 non-null  int64
 16  Breed1_7      10537 non-null  int64
 17  Color1_0      10537 non-null  int64
 18  Color1_1      10537 non-null  int64
 19  Color1_2      10537 non-n

In [19]:
for column in train.drop(columns = ['Target']).columns:
  mean = train[column].mean() #To-Do (Mean of train_data[column])
  std = train[column].std() #To-Do (Standard Deviation of train_data[column])
  train[column] = (train[column]- mean)/std
  test[column] = (test[column] - mean)/std

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns = ['Target']), train['Target'], test_size = 0.1)

print('Train examples: ', len(X_train), len(y_train))
print('Validation examples: ', len(X_valid), len(y_valid))
print('Test examples: ',  len(test))

Train examples:  9483 9483
Validation examples:  1054 1054
Test examples:  1000


In [21]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(26,)))
model.add(keras.layers.Dense(5000, activation='relu'))
model.add(keras.layers.Dense(1000, activation='relu'))
model.add(keras.layers.Dense(500, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

I0000 00:00:1742658861.711892   11897 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-22 19:24:21.713565: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 5000)           │       135,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1000)           │     5,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 500)            │       500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,637,001 (21.50 MB)

 Trainable params: 5,637,001 (21.50 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = ['accuracy'])

In [24]:
epochs = 10
BATCH_SIZE = 128

history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.7214 - loss: 0.5927 - val_accuracy: 0.7448 - val_loss: 0.5216
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.7505 - loss: 0.5203 - val_accuracy: 0.7505 - val_loss: 0.5126
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7534 - loss: 0.5033 - val_accuracy: 0.7581 - val_loss: 0.5045
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7608 - loss: 0.4983 - val_accuracy: 0.7514 - val_loss: 0.5140
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7727 - loss: 0.4794 - val_accuracy: 0.7552 - val_loss: 0.5033
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7752 - loss: 0.4777 - val_accuracy: 0.7457 - val_loss: 0.5056
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7799 - loss: 0.4650 - val_accuracy: 0.7505 - val_loss: 0.5134
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.7808 - loss: 0.4611 - val_accuracy: 0.7486 - v

In [25]:
from sklearn.metrics import f1_score
f1_score(y_valid, model.predict(X_valid) > 0.5, average= 'weighted')

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


0.7325826866861024

In [26]:
submission = pd.DataFrame(model.predict(test) > 0.5, columns = ['Target'])
submission

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


,Target
0,True
1,False
2,True
3,True
4,True
...,...
995,True
996,True
997,True
998,True
